In [1]:
import librosa
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
class AudioAugmentation():
      
    def __init__(self, sample_rate=None, num_samples=None):
        self.sample_rate = sample_rate
        self.num_samples = num_samples
    
    def read_audio_file(self, file_path):
        self.sample_rate = librosa.core.load(file_path)[1]
        self.num_samples = len(librosa.core.load(file_path)[0])
        samples = librosa.core.load(file_path)[0]
        return samples

    def write_audio_file(self, file, data):
        #librosa.output.write_wav(file, data, sample_rate)
        sf.write(file, data, self.sample_rate)

    def add_noise(self, data):
        noise = np.random.randn(len(data))
        data_noise = data + 0.005 * noise
        return data_noise

    def shift(self, data, seconds=1):
        return np.roll(data, int(self.sample_rate * seconds))

    def stretch(self, data, rate=1):
        data = librosa.effects.time_stretch(data, rate)
        if len(data) > self.num_samples:
            data = data[:self.num_samples]
        else:
            data = np.pad(data, (0, max(0, self.num_samples - len(data))), "constant")
        return data   


In [3]:
# Create a new instance from AudioAugmentation class
aa = AudioAugmentation()

a=0
duration = 1
barks_folder = 'barks_' + str(duration) + 's'
barks_path = os.path.join(os.getcwd(), barks_folder)
for file in os.listdir(barks_path):
    a+=1   
    file_path = os.path.join(barks_path, file)
    #print("file: " + file)
    
    # Read the cough sound
    data = aa.read_audio_file(file_path)
    #print("data: " + str(len(data)))
  
    # Adding noise to sound
    data_noise = aa.add_noise(data)
    #print("data_noise: " + str(len(data_noise)))
    
    # Shifting the sound
    data_roll = aa.shift(data)
    #print("data_roll: " + str(len(data_roll)))
  
    # Stretching the sound
    data_stretch = aa.stretch(data, 0.8)
    #print("data_stretch: " + str(len(data_stretch)))
    #print("- - - - - - - - - - -")
    
    output_folder = os.path.join(os.getcwd(), 'augments')
    
    noisefilename = "sample_noise_" + str(a) + ".wav"
    noisepath = os.path.join(output_folder, noisefilename)

    rollfilename = "sample_roll_" + str(a) + ".wav"
    rollpath = os.path.join(output_folder, rollfilename)

    stretchfilename = "sample_stretch_" + str(a) + ".wav"
    stretchpath = os.path.join(output_folder, stretchfilename)

    # Write generated cough sounds
    aa.write_audio_file(noisepath, data_noise)
    aa.write_audio_file(rollpath, data_roll)
    aa.write_audio_file(stretchpath, data_stretch)

In [4]:
# check num_samples is the same for all files
for file in (os.listdir(barks_path)):
    data = librosa.core.load(barks_path + '/' + file)[0]
    if len(data) != 22050 * duration:
        print(file)